equal width binning: 

if **n -> `no of bins`** <br>
now we use the formula:
<br>
`(max - min) / n`

- helps to handle outiers
- no change in spread of data

equal frequency binning: 
if **n -> `no of intervals`** <br>
each interval contains 10% of the data

eg : between 0-16 yrs 10% of the population resides <br>
16-20 -> 10% and so on

no equal width

K-means binning

-> makes clusters and used when data are present as clusters (***** ............. *****, ***** ......... *****)